In [1]:
index_cd = 'KPI200'

In [2]:
page_n = 1

In [3]:
naver_index ='http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)

In [4]:
from urllib.request import urlopen #인터넷에서 소스코드를 긁어오는 기능
source = urlopen(naver_index).read()

In [5]:
import bs4
source = bs4.BeautifulSoup(source,'lxml') 
#소스명.BeautifulSoup : 태그별로 분류

In [6]:
print(source.prettify())

<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   네이버 금융
  </title>
  <link href="/css/newstock.css?20190328184311" rel="stylesheet" type="text/css"/>
  <link href="/css/common.css?20190328184311" rel="stylesheet" type="text/css"/>
  <link href="/css/layout.css?20190328184311" rel="stylesheet" type="text/css"/>
  <link href="/css/main.css?20190328184311" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock2.css?20190328184311" rel="stylesheet" type="text/css"/>
  <link href="/css/newstock3.css?20190328184311" rel="stylesheet" type="text/css"/>
  <link href="/css/world.css?20190328184311" rel="stylesheet" type="text/css"/>
 </head>
 <body>
  <script src="/js/jindo.min.ns.1.5.3.euckr.js?20190328184311" type="text/javascript">
  </script>
  <script src="/js/lcslog.js?20190328184311" type="text/javascript">
  </script>
  <!-- 일별시세 -->
  <div class="box_type_m">
   <h4 class="top_tlt" style="text-align:left;">
    <em>
   

In [7]:
td = source.find_all('td') #find_all : 찾아낸 데이터를 리스트형태로 저장
len(td)

54

갯수가 54개인데 일일이 확인하기에는 어려움이 따름

데이터를 대량으로 제공하는 사이트에서 원하는 데이터의 위치를 찾아낼 때는 XPath주소를 이용하면 편리

XPath(XML Path Language) : 웹사이트 또는 XML 문서에 있는 각 항목의 주소를 문서에 포함된 태그를 조합한 경로형태로 표현하는 언어

In [8]:
#/html/body/div/table[1]/tbody/tr[3]/td[1]
#앞에서부터 슬래시 단위로 찾아가라는 뜻
source.find_all('table')[0].find_all('tr')[2].find_all('td')[0]

<td class="date">2019.04.04</td>

In [9]:
#<td class="date">처럼 class 이름을 이용해 특별히 이름을 붙인 td태그는 이름을 지정해서 뽑을 수 있음
d=source.find_all('td', class_='date')[0].text
d

'2019.04.04'

파이썬이 날짜를 인식할 수 있도록 형식 바꿔주기
점(.)으로 구분된 년,월,일을 분리하고 이를 datetime 라이브러리를 이용해 date형식으로 바꿔줌

In [10]:
import datetime as dt

문자열.split(구분자)함수를 이용해 구분자를 기준으로 문자열을 년,월,일로 분해

분해한 문자열은 dt.date(년,월,일)함수를 이용해 날짜 형식으로 바꿔서 this_date함수에 저장

In [11]:
yyyy = int(d.split('.')[0])
mm = int(d.split('.')[1])
dd = int(d.split('.')[2])
this_date = dt.date(yyyy,mm,dd)
this_date

datetime.date(2019, 4, 4)

읽어온 날짜 정보를 date형식으로 바꿀 일이 계속 생기므로 이 기능을 함수로 정의

문자열.replace('A','B') : 'A'를 'B'로 바꿔주는 기능

In [12]:
def date_format(d):
    d = str(d).replace('-','.') #날짜 구분자가 .이 아니라 -로 되어있을 경우, .으로 변화환해주는 기능
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy, mm, dd)
    return this_date

해당일자의 종가지수 가져오기

XPath = /html/body/div/table[1]/tbody/tr[3]/td[2]

종가를 가져올 때는 받아온 데이터를 숫자로 인식하도록 처리하기 위해 숫자에서 천의자리를 표시하는 쉼표(,)를 제거하는 작업이 필요

쉼표 제거에도 replace 함수 사용

In [13]:
this_close = source.find_all('tr')[2].find_all('td')[1].text
this_close = this_close.replace(',','') #쉼표 제거
this_close = float(this_close)
this_close

285.42

In [14]:
#<td class="number_1">285.65</td> 간단하게 나타내는 법
p = source.find_all('td', class_='number_1')[0].text
p

'285.42'

In [15]:
dates = source.find_all('td', class_='date')
prices = source.find_all('td', class_='number_1')

In [16]:
len(dates)

6

In [17]:
len(prices)

24

'체결가','등락률','거래량','거래대금' 칼럼이 모두 같은 태그를 사용

4개의 값 중 첫 번째 값만 필요하므로 0, 4, 8, 등 4의 배수로 나타나는 값들만 추출

<현재 페이지에서 날짜와 종가지수 추출하는 코드>

In [18]:
for n in range(len(dates)):#dates 개수만큼 반복
    this_date = dates[n].text #n번째 dates 값 추출
    this_date = date_format(this_date) #날짜 형식으로 변환
    
    this_close = prices[n*4].text #0, 4, 8,... 등 4의 배수에 해당하는 종가지수 추출
    this_close = this_close.replace(',','') #쉼표 제거
    this_close = float(this_close) #숫자 형식으로 변환
    this_close
    
    print(this_date, this_close) #결과값 출력

2019-04-04 285.42
2019-04-03 285.65
2019-04-02 281.72
2019-04-01 280.32
2019-03-29 276.48
2019-03-28 275.08


In [19]:
#마지막 링크를 가르키는 하이퍼링크 <a>태그는 <td class="pgRR"> 태그
paging = source.find('td', class_='pgRR').find('a')['href']
paging

'/sise/sise_index_day.nhn?code=KPI200&page=546'

뽑아온 하이퍼링크 주소에서 페이지 번호인 546만 추출하기 위해 &을 기준으로 문자열을 잘라내 뒷부분만 가져온다.

In [20]:
paging = paging.split('&')[1]
paging

'page=546'

In [21]:
paging = paging.split('=')[1]
paging

'546'

뽑아온 페이지 번호를 숫자 형식으로 바꿔준다.

마지막 페이지 번호를 뽑는 과정

In [22]:
last_page = source.find('td', class_='pgRR').find('a')['href']
last_page = last_page.split('&')[1]
last_page = last_page.split('=')[1]
last_page = int(last_page)

In [23]:
last_page

546

<h3>데이터 추출 기능을 함수로 만들기</h3>

현재 페이지에서 데이터를 추출하는 작업을 페이지마다 반복해야하기 때문에 함수로 만들어서 활용

In [24]:
def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    
    if start_date: #start_date가 있으면
        start_date = date_format(start_date) #date 포맷으로 변환
    else: #없으면
        start_date = dt.date.today() #오늘 날짜를 지정
    if end_date:
        end_date = date_format(end_date)
    else:
        end_date = dt.date.today()
    
    naver_index = 'https://finance.naver.com/sise/sise_index_day.nhn?code='+index_cd+'&page='+str(page_n)
    
    source = urlopen(naver_index).read() #저장한 페이지에서 코드 읽기
    source = bs4.BeautifulSoup(source,'lxml') #뷰티플 수프로 태그별로 코드 분류
    
    dates = source.find_all('td', class_='date') #<td class="date"> 태그에서 날짜 수집
    prices = source.find_all('td', class_='number_1') #<td class="number_1">태그에서 지수 수집
    
    for n in range(len(dates)):
        if dates[n].text.split('.')[0].isdigit():
            #날짜 처리
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
            #start_date와 end_date 사이에서 데이터 저장
                #종가처리
                this_close = prices[n*4].text #prices 중 종가지수인 0, 4, 8... 번째 데이터 추출
                this_close = this_close.replace(',','')
                this_close = float(this_close)

                #딕셔너리에 저장
                historical_prices[this_date] = this_close
                
            elif this_date < start_date:
            #start_date 이전이면 함수 종료
            #도달하자마자 함수 실행을 종료하면 추가적인 데이터 탐색 없이 효율적으로 데이터를 수집할 수 있음
                return historical_prices

    #페이지 네비게이션
    if last_page == 0:
        last_page = source.find('td', class_='pgRR').find('a')['href']
        #마지막 페이지 주소 추출
        last_page = last_page.split('&')[1] # & 뒤의 page = 536 부분 추출
        last_page = last_page.split('=')[1] # '=' 뒤의 페이지 번호만 추출
        last_page = int(last_page) # 숫자형 변수로 변환
        
    #다음 페이지 호출
    if page_n < last_page:
        page_n = page_n + 1
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)
        
    return historical_prices

날짜를 지정해서 추출하는 함수 -> 함수에서 시작 날짜와 마지막 날짜를 입력시켜 데이터 추출을 제어

In [25]:
index_cd = 'KPI200'
historical_prices = dict()
historical_index_naver(index_cd, '2019-4-1','2019-4-4')
historical_prices

{datetime.date(2019, 4, 1): 280.32,
 datetime.date(2019, 4, 2): 281.72,
 datetime.date(2019, 4, 3): 285.65,
 datetime.date(2019, 4, 4): 285.42}